In [5]:
# import modules

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.sparse as sp

import pandas as pd

from itertools import combinations, permutations, product

In [6]:
# change settings for the graphs

fsize = 7
tsize = 9
tdir = 'in'
major = 5.0
minor = 3.0
lwidth = 0.8
lhandle = 2.0
plt.style.use('default')
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = tsize
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['axes.linewidth'] = lwidth
plt.rcParams['legend.handlelength'] = lhandle

In [7]:
#functions for generating the basis states in a 2 leg ladder geometry

def blockade_ladder(L):
    if L == 1:
        return [[0,0], [0,1], [1,0]]
    
    subreps = blockade_ladder(L-1)

    greps = [[0,0] + r for r in subreps]
    ureps = [[0,1] + r for r in subreps if r[1] != 1]
    dreps = [[1,0] + r for r in subreps if r[0] != 1]

    reps = greps + ureps + dreps

    return reps

def matrix_blockade_ladder(L):

    if L == 1:
        return [[0,0], [0,1], [1,0]]

    subreps = blockade_ladder(L-1)

    greps = [[0,0] + r for r in subreps]
    ureps = [[0,1] + r for r in subreps if r[1] != 1 and r[-1] != 1]
    dreps = [[1,0] + r for r in subreps if r[0] != 1 and r[-2] != 1]

    reps = greps + ureps + dreps

    #horizontal_reps = [np.hstack(np.transpose(np.reshape(i, (L,2)))) for i in vertical_reps] # type: ignore
    matrix_reps = np.array([np.transpose(np.reshape(i, (L,2))) for i in reps]) # type: ignore

    return matrix_reps

def open_matrix_blockade_ladder(L):

    if L == 1:
        return [[0,0], [0,1], [1,0]]

    subreps = blockade_ladder(L-1)

    greps = [[0,0] + r for r in subreps]
    ureps = [[0,1] + r for r in subreps if r[1] != 1]
    dreps = [[1,0] + r for r in subreps if r[0] != 1]

    reps = greps + ureps + dreps

    #horizontal_reps = [np.hstack(np.transpose(np.reshape(i, (L,2)))) for i in vertical_reps] # type: ignore
    matrix_reps = np.array([np.transpose(np.reshape(i, (L,2))) for i in reps]) # type: ignore

    return matrix_reps

def full_ladder(L):
    if L == 1:
        return [[0,0], [0,1], [1,0], [1,1]]
    
    subreps = full_ladder(L-1)

    greps = [[0,0] + r for r in subreps]
    ureps = [[0,1] + r for r in subreps]
    dreps = [[1,0] + r for r in subreps]
    breps = [[1,1] + r for r in subreps]

    reps = greps + ureps + dreps + breps

    return reps

def matrix_full_ladder(L):
    subreps = full_ladder(L)

    #horizontal_reps = [np.hstack(np.transpose(np.reshape(i, (L,2)))) for i in vertical_reps] # type: ignore
    matrix_reps = np.array([np.transpose(np.reshape(i, (L,2))) for i in subreps]) # type: ignore

    return matrix_reps

In [8]:
# functions for generating the basis states for a one dimensional chain

def blockade_line(L):
    if L == 1:
        return [[0], [1]]
    
    subreps = blockade_line(L-1)

    greps = [[0] + r for r in subreps]
    ereps = [[1] + r for r in subreps if r[0] != 1]

    reps = greps + ereps

    return reps

def matrix_blockade_line(L):

    if L == 1:
        return [[0], [1]]

    subreps = blockade_line(L-1)

    greps = [[0] + r for r in subreps]
    ereps = [[1] + r for r in subreps if r[0] != 1 and r[-1] != 1]

    reps = greps + ereps

    matrix_reps = np.array([np.transpose(np.reshape(i, (L,1))) for i in reps]) # type: ignore

    return matrix_reps

def open_matrix_blockade_line(L):

    if L == 1:
        return [[0], [1]]

    subreps = blockade_line(L-1)

    greps = [[0] + r for r in subreps]
    ereps = [[1] + r for r in subreps if r[0] != 1]

    reps = greps + ereps

    matrix_reps = np.array([np.transpose(np.reshape(i, (L,1))) for i in reps]) # type: ignore

    return matrix_reps

def full_line(L):
    if L == 1:
        return [[0], [1]]
    
    subreps = full_line(L-1)

    greps = [[0] + r for r in subreps]
    ereps = [[1] + r for r in subreps]

    reps = greps + ereps

    return reps

def matrix_full_line(L):

    if L == 1:
        return [[0], [1]]

    subreps = full_line(L-1)

    greps = [[0] + r for r in subreps]
    ereps = [[1] + r for r in subreps]

    reps = greps + ereps

    matrix_reps = np.array([np.transpose(np.reshape(i, (L,1))) for i in reps]) # type: ignore

    return matrix_reps


In [9]:
# functions for sorting and splitting the basis states by the number of excitations

def sort(basis):
    sorted_basis = np.array(sorted(basis, key = lambda x: np.sum(x)))
    n_e_max = np.flip(basis)[0].sum()
    n_e, bin_edges = np.histogram([(x.sum(axis=0)).sum(axis=0) for x in sorted_basis], bins=n_e_max+1)
    edges = np.cumsum(n_e)#[:-1]
    return sorted_basis, edges

def split(sorted_basis, edges):
    split_basis = np.split(sorted_basis, edges)
    return split_basis

$$
H_n=\sum_i \hat{n}_i
$$

In [10]:
def detuning_hamiltonian(basis):

    sorted_basis, edges = sort(basis)
    dim = len(sorted_basis)

    diag = [np.sum(i) for i in sorted_basis] # 2L sites in 2 leg ladder

    mat = sp.dia_array((diag,0), shape=(dim, dim))

    return mat

$$ H_\Delta = \sum_i (-1)^i \sigma_i^z

In [11]:
def staggered_detuning_hamiltonian(basis):

    sorted_basis, edges = sort(basis)
    dim = len(sorted_basis)

    diag0 = np.array([2*np.sum(i[0][::2]) - len(i[0][::2]) - (2*np.sum(i[0][1::2]) - len(i[0][1::2])) for i in sorted_basis]) # even - odd
    if len(sorted_basis[0]) == 2:
        diag1 = np.array([2*np.sum(i[1][::2]) - len(i[1][::2]) - (2*np.sum(i[1][1::2]) - len(i[1][1::2])) for i in sorted_basis]) # even - odd
    else:
        diag1 = np.zeros(dim)
    mat = sp.dia_array((diag0 + diag1,0), shape=(dim, dim))

    return mat

$$ H_\Omega = \sum_i \sigma_i^x

In [12]:
def rabi_hamiltonian(basis):

    sorted_basis, edges = sort(basis)
    split_basis = split(sorted_basis, edges)

    edges = np.append(0, edges[:-1])
    dim = len(sorted_basis)
    L = len(sorted_basis[0][0])
    legs = len(sorted_basis[0])

    empty = sp.coo_array((dim, dim))
    r_mat = np.full((legs, L), empty)

    for count, set in enumerate(split_basis):
        for x, state in enumerate(set):
            compare = state == split_basis[count+1]
            matches = np.sum(np.sum(compare, axis=1), axis=1)
            couplings = np.where(matches == legs*L-1)[0]

            for y in couplings:
                error = np.where(compare[y] == 0)
                r_mat[error[0][0], error[1][0]] += sp.coo_array(([1], ([edges[count] + x], [edges[count + 1] + y])), shape=(dim,dim))
                r_mat[error[0][0], error[1][0]] += sp.coo_array(([1], ([edges[count + 1] + y], [edges[count] + x])), shape=(dim,dim))

    mat = np.sum(r_mat)

    return mat, r_mat

$$H_t = \sum_{\langle ij\rangle} \left( b_i^\dagger b_j + \text{h.c.}\right)

In [13]:
def hopping_hamiltonian(basis, bc):

    sorted_basis, edges = sort(basis)
    split_basis = split(sorted_basis, edges)

    edges = np.append(0, edges[:-1])
    dim = len(sorted_basis)
    L = len(sorted_basis[0][0])
    
    if bc == 'PBC':
        l = len(sorted_basis[0][0])
    elif bc == 'OBC':
        l = 0

    legs = len(sorted_basis[0])

    empty = sp.coo_array((dim, dim))

    h_mat = np.full((legs,L), empty)
    v_mat = np.full((legs,L), empty)
    d_mat = np.full((legs,L), empty)

    for count, set in enumerate(split_basis):
        for x, state in enumerate(set):
            compare = state == set
            matches = np.sum(np.sum(compare, axis=1), axis=1)
            possible_couplings = np.where(matches == legs*L-2)[0]
            #print(compare)
            couplings = []

            for y in possible_couplings:
                errors = np.where(compare[y] == 0)
                diff = np.diff(errors)
                if np.all(diff == [[0],[1]]) or np.all(diff == [[0],[-(l-1)]]): # Horizontal
                    h_mat[errors[0][0]][errors[1][0]] += sp.coo_array(([1], ([edges[count] + x], [edges[count] + y])), shape=(dim,dim))

                elif np.all(diff == [[0],[0]]) or np.all(diff == [[0],[l-1]]): # Horizontal
                    h_mat[errors[0][1]][errors[1][1]] += sp.coo_array(([1], ([edges[count] + x], [edges[count] + y])), shape=(dim,dim))

                elif np.all(diff == [[1],[0]]): # Vertical
                    v_mat[errors[0][0]][errors[1][0]] += sp.coo_array(([1], ([edges[count] + x], [edges[count] + y])), shape=(dim,dim))

                elif np.all(diff == [[-1],[0]]): # Vertical
                    v_mat[errors[0][1]][errors[1][1]] += sp.coo_array(([1], ([edges[count] + x], [edges[count] + y])), shape=(dim,dim))

                elif np.all(diff == [[1],[-1]]) or np.all(diff == [[-1],[-1]]) or np.all(diff == [[1],[(l-1)]]) or np.all(diff == [[-1],[(l-1)]]): # Diagonal
                    d_mat[errors[0][1]][errors[1][1]] += sp.coo_array(([1/(2**1.5)], ([edges[count] + x], [edges[count] + y])), shape=(dim,dim))

                elif np.all(diff == [[1],[1]]) or np.all(diff == [[-1],[1]]) or np.all(diff == [[1],[-(l-1)]]) or np.all(diff == [[-1],[-(l-1)]]): # Diagonal
                    d_mat[errors[0][0]][errors[1][0]] += sp.coo_array(([1/(2**1.5)], ([edges[count] + x], [edges[count] + y])), shape=(dim,dim))

    mat = np.sum(h_mat) + np.sum(v_mat) + np.sum(d_mat)

    return mat, h_mat, v_mat, d_mat

In [14]:
def interaction_strength(state, L, bc):
    potential = []

    for combo in combinations(np.transpose(np.where(state)), 2):  # 2 for pairs, 3 for triplets, etc
        diff = np.linalg.norm(np.diff(np.reshape(combo, (2,2)).T))
        # if diff == 1 or (diff == L-1 and bc=='PBC'):
        #     potential.append(1)
        potential.append(1/diff**6)

    return np.sum(potential)

def interaction_hamiltonian(basis, bc):

    sorted_basis, edges = sort(basis)

    L = len(sorted_basis[0][0])

    dim = len(sorted_basis)

    diag = []

    for i in sorted_basis:
        diag.append(interaction_strength(i, L, bc))

    mat = sp.dia_array((diag,0), shape=(dim, dim))

    return mat

In [15]:
# calculates the matrices, possibly degenerate, ground state.
# very inefficient for highly degenerate states

def groundstate(mat):

    num_states = 1
    E, GS = sp.linalg.eigsh(mat.mat, k=num_states, which='SA')

    while num_states < len(mat.basis) - 1:

        num_states += 1

        new_E, new_GS = sp.linalg.eigsh(mat.mat, k=num_states, which='SA')

        gaps = np.diff(new_E)
        max_gap = np.max(gaps)

        if max_gap > 1e-10:
            break

        # limit maximum gap

        E = new_E
        GS = new_GS

    return np.round(E, 10), np.sum(GS, axis=1)/np.sqrt(num_states - 1), num_states - 1


In [16]:
# defines the van neumann entropy of a given ground state

def S_vn(GS,N,N_l,N_r):
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    plt.rcParams['font.size'] = '16'
    GS = GS.reshape((2 ** N, 1))

    n1, n2, n = 2 ** N_l, 2 ** N_r, 2 ** N
    rho = GS @ np.conj(GS.T)
    rho = rho / np.trace(rho)
    rho_tensor = rho.reshape([n1, n2, n1, n2])
    # np.trace(rho_tensor, axis1=1, axis2=3)  # rho_a
    rho_a = np.trace(rho_tensor, axis1=0, axis2=2)
    # print(rho_a.shape)
    pn = np.real(np.linalg.eigvalsh(rho_a))
    n = pn > 0
    if len(n) > 0:
        # print("IF")
        S = -np.sum(pn[n] * np.log(pn[n]))
    else:
        # print("Else")
        S = 0.0  # p log p = 0 if p = 0

    if S == 0:
        S = abs(S)

    return np.round(S,4)

In [17]:
class lattice:
    def __init__(self, sites, bc='OBC', legs=1, constraint=True) -> None:
        self.sites = sites

        if legs == 1:
            if constraint == True:
                if bc == 'PBC':
                    self.basis = sort(matrix_blockade_line(self.sites))[0]
                elif bc == 'OBC':
                    self.basis = sort(open_matrix_blockade_line(self.sites))[0]
            else:
                self.basis = sort(matrix_full_line(self.sites))[0]
            
        elif legs == 2:
            if constraint == True:
                if bc == 'PBC':
                    self.basis = sort(matrix_blockade_ladder(self.sites))[0]
                elif bc == 'OBC':
                    self.basis = sort(open_matrix_blockade_ladder(self.sites))[0]
            else:
                self.basis = sort(matrix_full_ladder(self.sites))[0]

        self.det = detuning_hamiltonian(self.basis)
        self.stag = staggered_detuning_hamiltonian(self.basis)
        self.rabi, self.r_rabi = rabi_hamiltonian(self.basis)
        self.hop, self.h_hop, self.v_hop, self.d_hop = hopping_hamiltonian(self.basis, bc)
        self.int = interaction_hamiltonian(self.basis, bc)
    
class matrix(lattice):
    def __init__(self, lattice, det=0.0, stag=0.0, rabi=0.0, hop=0.0, int=0.0) -> None:
        self.sites = lattice.sites
        self.basis = lattice.basis

        self.det, self.stag, self.rabi, self.hop, self.int = det, stag, rabi, hop, int

        self.mat = det * lattice.det + stag * lattice.stag + rabi * lattice.rabi + hop * lattice.hop + int * lattice.int
        self.energy, self.gs, self.degen = groundstate(self)
        #self.dist = np.dot(self.cont.transpose(), np.reshape(self.basis, (len(self.basis), 2*self.sites))).transpose()
        #self.dens = np.sum(self.dist) / (2 * self.sites)
    # def gs(self):
    #     return np.vectorize(GS(self))


In [18]:
'''
lat = lattice(15)

fig, ax = plt.subplots(ncols=2, figsize=(7,3), dpi=200)

edges = sort(lat.basis)[1]

ax[0].spy(lat.rabi, markersize=.0001)
ax[0].set_title('Rabi Term', size=11)

ax[1].spy(lat.hop, markersize=.0001)
ax[1].set_title('Hopping Term', size=11)

for i in range(2):
    ax[i].hlines(edges, 0, np.shape(lat.rabi)[0], linewidth=.7, color='tab:gray')
    ax[i].vlines(edges, 0, np.shape(lat.rabi)[0], linewidth=.7, color='tab:gray')
    ax[i].set_xlabel(r'$j$', size=9)
    ax[i].set_ylabel(r'$i$', size=9)
    ax[i].tick_params(top=False, labeltop=False, labelbottom=True)
'''

"\nlat = lattice(15)\n\nfig, ax = plt.subplots(ncols=2, figsize=(7,3), dpi=200)\n\nedges = sort(lat.basis)[1]\n\nax[0].spy(lat.rabi, markersize=.0001)\nax[0].set_title('Rabi Term', size=11)\n\nax[1].spy(lat.hop, markersize=.0001)\nax[1].set_title('Hopping Term', size=11)\n\nfor i in range(2):\n    ax[i].hlines(edges, 0, np.shape(lat.rabi)[0], linewidth=.7, color='tab:gray')\n    ax[i].vlines(edges, 0, np.shape(lat.rabi)[0], linewidth=.7, color='tab:gray')\n    ax[i].set_xlabel(r'$j$', size=9)\n    ax[i].set_ylabel(r'$i$', size=9)\n    ax[i].tick_params(top=False, labeltop=False, labelbottom=True)\n"

In [19]:
# returns the overlap of a specific matrices ground state with the implemented ansatz

'''

def ansatzmatch(L):
    lat = lattice(L, legs=1, bc='OBC', constraint=True)
    mat = matrix(lat, rabi=2, hop=1)
    
    A = np.array([1, 0])
    B = np.array([1, -1]) / np.sqrt(2)
    
    s = np.kron(np.kron(np.kron(A, B), A), B)
    
    def ansatz1(L):
        if L == 1:
            return A
        ansatz = ansatz2(L-1)
        return np.kron(A, ansatz)
    
    def ansatz2(L):
        if L == 1:
            return B
        ansatz = ansatz1(L-1)
        return np.kron(B, ansatz)
    
    restricted = np.array([int(''.join(map(str, lat.basis[i][0])), 2) for i in range(len(lat.basis))])
    
    #Ansatz1 = ansatz1(L)[restricted]
    Ansatz2 = ansatz2(L)[restricted]

    return np.abs(Ansatz2.T @ mat.gs)**2, np.abs(Ansatz2.T @ mat.gs)**2 - 1

#binary = np.array([[int(bit) for bit in f"{i:0{L}b}"] for i in range(len(s))])

#for i in range(len(s)):
#    print(s[i], ':', [int(bit) for bit in f"{i:0{L}b}"])
'''

'\n\ndef ansatzmatch(L):\n    lat = lattice(L, legs=1, bc=\'OBC\', constraint=True)\n    mat = matrix(lat, rabi=2, hop=1)\n    \n    A = np.array([1, 0])\n    B = np.array([1, -1]) / np.sqrt(2)\n    \n    s = np.kron(np.kron(np.kron(A, B), A), B)\n    \n    def ansatz1(L):\n        if L == 1:\n            return A\n        ansatz = ansatz2(L-1)\n        return np.kron(A, ansatz)\n    \n    def ansatz2(L):\n        if L == 1:\n            return B\n        ansatz = ansatz1(L-1)\n        return np.kron(B, ansatz)\n    \n    restricted = np.array([int(\'\'.join(map(str, lat.basis[i][0])), 2) for i in range(len(lat.basis))])\n    \n    #Ansatz1 = ansatz1(L)[restricted]\n    Ansatz2 = ansatz2(L)[restricted]\n\n    return np.abs(Ansatz2.T @ mat.gs)**2, np.abs(Ansatz2.T @ mat.gs)**2 - 1\n\n#binary = np.array([[int(bit) for bit in f"{i:0{L}b}"] for i in range(len(s))])\n\n#for i in range(len(s)):\n#    print(s[i], \':\', [int(bit) for bit in f"{i:0{L}b}"])\n'

In [20]:
# returns the excitation densities for different particle numbers and three combinations of omega and t

'''
fig, ax = plt.subplots(nrows=3, ncols=2, sharex='col', sharey='all', dpi=150)

N1 = 15
N2 = 16

lat = [lattice(N1, legs=1, bc='OBC'), lattice(N2, legs=1, bc='OBC')]
rabi = [1, 2, 0]
hop = [0, 1, 1]

for i in range(2):
    for j in range(3):
        mat = matrix(lattice=lat[i], rabi=rabi[j], hop=hop[j])
        density = [np.sum([mat.gs[k]**2 * mat.basis[k][0,x] for k in range(len(mat.basis))]) for x in range(mat.sites)]
        ax[j,i].plot(density)
        ax[j,i].scatter(range(len(density)), density, s=7)

ax[0,0].set_title(f'N = {N1}', size=12)
ax[0,1].set_title(f'N = {N2}', size=12)

ax[0,0].set_ylabel(r'$\Omega \gg t$', size=12)
ax[1,0].set_ylabel(r'$4\Omega = t$', size=12)
ax[2,0].set_ylabel(r'$\Omega \ll t$', size=12)
ax[1,0].set_ylim(-0.05, .55)

ax[2,0].set_xlabel('Site index', size=9)
ax[2,1].set_xlabel('Site index', size=9)

ax[1,0].set_yticks([0,0.1,0.2,0.3,0.4,0.5], labels=[0,0.1,0.2,0.3,0.4,0.5])

#fig.suptitle('Test')
fig.supylabel('Excitation density', size=9)
'''

"\nfig, ax = plt.subplots(nrows=3, ncols=2, sharex='col', sharey='all', dpi=150)\n\nN1 = 15\nN2 = 16\n\nlat = [lattice(N1, legs=1, bc='OBC'), lattice(N2, legs=1, bc='OBC')]\nrabi = [1, 2, 0]\nhop = [0, 1, 1]\n\nfor i in range(2):\n    for j in range(3):\n        mat = matrix(lattice=lat[i], rabi=rabi[j], hop=hop[j])\n        density = [np.sum([mat.gs[k]**2 * mat.basis[k][0,x] for k in range(len(mat.basis))]) for x in range(mat.sites)]\n        ax[j,i].plot(density)\n        ax[j,i].scatter(range(len(density)), density, s=7)\n\nax[0,0].set_title(f'N = {N1}', size=12)\nax[0,1].set_title(f'N = {N2}', size=12)\n\nax[0,0].set_ylabel(r'$\\Omega \\gg t$', size=12)\nax[1,0].set_ylabel(r'$4\\Omega = t$', size=12)\nax[2,0].set_ylabel(r'$\\Omega \\ll t$', size=12)\nax[1,0].set_ylim(-0.05, .55)\n\nax[2,0].set_xlabel('Site index', size=9)\nax[2,1].set_xlabel('Site index', size=9)\n\nax[1,0].set_yticks([0,0.1,0.2,0.3,0.4,0.5], labels=[0,0.1,0.2,0.3,0.4,0.5])\n\n#fig.suptitle('Test')\nfig.supylabel('

In [21]:
# same as above, but for only one number particles

'''
L = 21

lat = lattice(L, legs=1, bc='OBC', constraint=True)

fig, ax = plt.subplots(nrows=1, ncols=3, dpi=200, figsize=(10, 2))

rabi = [1, 2, 0]
hop = [0, 1, 1]

for j in range(3):
    mat = matrix(lattice=lat, rabi=rabi[j], hop=hop[j], int=0)
    density = [np.sum([mat.gs[k]**2 * mat.basis[k][0,x] for k in range(len(mat.basis))]) for x in range(mat.sites)]
    ax[j].plot(density)
    ax[j].scatter(range(len(density)), density, s=7)

ax[0].set_title(r'$\Omega \gg t$', size=12)
ax[1].set_title(r'$\Omega = 4t$', size=12)
ax[2].set_title(r'$\Omega \ll t$', size=12)
ax[0].set_ylim(-0.05, .55)
ax[1].set_ylim(-0.05, .55)
ax[2].set_ylim(-0.05, .55)

ax[0].set_xlabel('Site index', size=9)
ax[1].set_xlabel('Site index', size=9)
ax[2].set_xlabel('Site index', size=9)

ax[0].set_yticks([0,0.1,0.2,0.3,0.4,0.5], labels=[0,0.1,0.2,0.3,0.4,0.5])
ax[0].set_xticks(np.arange(0, L, int(np.round(L/5))), labels=np.arange(0, L, int(np.round(L/5))))
ax[1].set_xticks(np.arange(0, L, int(np.round(L/5))), labels=np.arange(0, L, int(np.round(L/5))))
ax[2].set_xticks(np.arange(0, L, int(np.round(L/5))), labels=np.arange(0, L, int(np.round(L/5))))

# fig.suptitle('Test')
ax[0].set_ylabel('Excitation density', size=9)
'''

"\nL = 21\n\nlat = lattice(L, legs=1, bc='OBC', constraint=True)\n\nfig, ax = plt.subplots(nrows=1, ncols=3, dpi=200, figsize=(10, 2))\n\nrabi = [1, 2, 0]\nhop = [0, 1, 1]\n\nfor j in range(3):\n    mat = matrix(lattice=lat, rabi=rabi[j], hop=hop[j], int=0)\n    density = [np.sum([mat.gs[k]**2 * mat.basis[k][0,x] for k in range(len(mat.basis))]) for x in range(mat.sites)]\n    ax[j].plot(density)\n    ax[j].scatter(range(len(density)), density, s=7)\n\nax[0].set_title(r'$\\Omega \\gg t$', size=12)\nax[1].set_title(r'$\\Omega = 4t$', size=12)\nax[2].set_title(r'$\\Omega \\ll t$', size=12)\nax[0].set_ylim(-0.05, .55)\nax[1].set_ylim(-0.05, .55)\nax[2].set_ylim(-0.05, .55)\n\nax[0].set_xlabel('Site index', size=9)\nax[1].set_xlabel('Site index', size=9)\nax[2].set_xlabel('Site index', size=9)\n\nax[0].set_yticks([0,0.1,0.2,0.3,0.4,0.5], labels=[0,0.1,0.2,0.3,0.4,0.5])\nax[0].set_xticks(np.arange(0, L, int(np.round(L/5))), labels=np.arange(0, L, int(np.round(L/5))))\nax[1].set_xticks(np.ar

In [22]:
# generates the phase diagram for varying detuning and interaction strength

'''
L = 13

lat = lattice(L, legs=1, bc='OBC', constraint=False)

X, Y = np.meshgrid(
    np.linspace(1,5,40),
    np.linspace(1,5,40) # y_max = 4 ?
)

print(X.shape)

Z = np.zeros_like(X)
for x in range(X.shape[0]):
    print(x)
    for y in range(Y.shape[1]):
        print(y)
        try:
            # print(X[0,x], Y[y,0])
            mat = matrix(lat, det=-X[0,x], rabi=.5, int=Y[y,0]**6)
            # z = np.sum(mat.basis[:,0,:].T * mat.gs**2)
            z = S_vn(mat.gs, L,L//2+L%2,L//2)
        except:
            pass
        else:
            Z[x,y] = z
'''

#np.save('phase55.npy', Z)

"\nL = 13\n\nlat = lattice(L, legs=1, bc='OBC', constraint=False)\n\nX, Y = np.meshgrid(\n    np.linspace(1,5,40),\n    np.linspace(1,5,40) # y_max = 4 ?\n)\n\nprint(X.shape)\n\nZ = np.zeros_like(X)\nfor x in range(X.shape[0]):\n    print(x)\n    for y in range(Y.shape[1]):\n        print(y)\n        try:\n            # print(X[0,x], Y[y,0])\n            mat = matrix(lat, det=-X[0,x], rabi=.5, int=Y[y,0]**6)\n            # z = np.sum(mat.basis[:,0,:].T * mat.gs**2)\n            z = S_vn(mat.gs, L,L//2+L%2,L//2)\n        except:\n            pass\n        else:\n            Z[x,y] = z\n"

In [23]:
# displays said phase diagram with additional labels

'''
X, Y = np.meshgrid(
    np.linspace(1,5,40),
    np.linspace(1,5,40) # y_max = 4 ?
)

phase = np.load("phase55.npy")

fig, ax = plt.subplots(figsize=(8,5), dpi=200)

c = ax.pcolormesh(X, Y, phase.T, cmap='inferno')
ax.set_xlabel(r'$\frac{\Delta}{\Omega}$', size=15)
ax.set_ylabel(r'$\frac{R_b}{a}$', size=15)

ax.scatter([4, 4, 4],[1.5, 2.5, 3.5], c='white', s=1)
# ax.text(3+.05, 0.5-.1, r'$Z_1$', c='white', size=13)
ax.text(4+.05, 1.5-.07, r'$Z_2$', c='white', size=15)
ax.text(4+.05, 2.5-.07, r'$Z_3$', c='white', size=15)
ax.text(4+.05, 3.5-.07, r'$Z_4$', c='white', size=15)

ax.tick_params(labelsize=11)

cb = fig.colorbar(c, ax=ax)
cb.ax.tick_params(labelsize=11)
cb.set_label(r'$S_{vN}$', size=13)
'''

'\nX, Y = np.meshgrid(\n    np.linspace(1,5,40),\n    np.linspace(1,5,40) # y_max = 4 ?\n)\n\nphase = np.load("phase55.npy")\n\nfig, ax = plt.subplots(figsize=(8,5), dpi=200)\n\nc = ax.pcolormesh(X, Y, phase.T, cmap=\'inferno\')\nax.set_xlabel(r\'$\x0crac{\\Delta}{\\Omega}$\', size=15)\nax.set_ylabel(r\'$\x0crac{R_b}{a}$\', size=15)\n\nax.scatter([4, 4, 4],[1.5, 2.5, 3.5], c=\'white\', s=1)\n# ax.text(3+.05, 0.5-.1, r\'$Z_1$\', c=\'white\', size=13)\nax.text(4+.05, 1.5-.07, r\'$Z_2$\', c=\'white\', size=15)\nax.text(4+.05, 2.5-.07, r\'$Z_3$\', c=\'white\', size=15)\nax.text(4+.05, 3.5-.07, r\'$Z_4$\', c=\'white\', size=15)\n\nax.tick_params(labelsize=11)\n\ncb = fig.colorbar(c, ax=ax)\ncb.ax.tick_params(labelsize=11)\ncb.set_label(r\'$S_{vN}$\', size=13)\n'

In [24]:
# calculate and show the excitation densities of the points marked in the above phase diagram

'''
L=13
lat = lattice(L, legs=1, bc='OBC', constraint=False)
mat = []

int = [1.5,2.5,3.5,4.5]

for j in range(4):
    mat.append(matrix(lattice=lat, det=-4, rabi=1/2, int=int[j]**6))

fig, ax = plt.subplots(nrows=2, ncols=2, dpi=200, figsize=(8,5))

for i,j in product([0,1],[0,1]):
    density = [np.sum([mat[2*i+j].gs[k]**2 * mat[2*i+j].basis[k][0,x] for k in range(len(mat[2*i+j].basis))]) for x in range(mat[2*i+j].sites)]
    ax[i,j].plot(density)
    ax[i,j].scatter(range(len(density)), density, s=7)
    
    ax[i,j].set_title(rf'$\frac{{R_b}}{{a}}={int[2*i+j]}$', size=12)
    ax[1,j].set_xlabel('Site index', size=9)
    ax[i,0].set_ylabel('Excitation density', size=9)
    ax[i,j].set_ylim(-0.05, 1.05)

# fig.savefig(fname='z_n.png', dpi=200)
'''

"\nL=13\nlat = lattice(L, legs=1, bc='OBC', constraint=False)\nmat = []\n\nint = [1.5,2.5,3.5,4.5]\n\nfor j in range(4):\n    mat.append(matrix(lattice=lat, num=-4, rabi=1/2, int=int[j]**6))\n\nfig, ax = plt.subplots(nrows=2, ncols=2, dpi=200, figsize=(8,5))\n\nfor i,j in product([0,1],[0,1]):\n    density = [np.sum([mat[2*i+j].gs[k]**2 * mat[2*i+j].basis[k][0,x] for k in range(len(mat[2*i+j].basis))]) for x in range(mat[2*i+j].sites)]\n    ax[i,j].plot(density)\n    ax[i,j].scatter(range(len(density)), density, s=7)\n    \n    ax[i,j].set_title(rf'$\x0crac{{R_b}}{{a}}={int[2*i+j]}$', size=12)\n    ax[1,j].set_xlabel('Site index', size=9)\n    ax[i,0].set_ylabel('Excitation density', size=9)\n    ax[i,j].set_ylim(-0.05, 1.05)\n\n# fig.savefig(fname='z_n.png', dpi=200)\n"

In [25]:
# same thing for two points in a different system

'''
lat1 = lattice(11, legs=1, bc='OBC', constraint=False)
lat2 = lattice(13, legs=1, bc='OBC', constraint=False)

mat = []
mat.append(matrix(lat1, det=-4, rabi=1/2, int=4.5**6))
mat.append(matrix(lat2, det=-4, rabi=1/2, int=5.5**6))

fig, ax = plt.subplots(nrows=1, ncols=2, dpi=200, figsize=(8,2.5))

title = [r'$Z_5$', r'$Z_6$']

for i in range(2):
    density = [np.sum([mat[i].gs[k]**2 * mat[i].basis[k][0,x] for k in range(len(mat[i].basis))]) for x in range(mat[i].sites)]
    ax[i].plot(density)
    ax[i].scatter(range(len(density)), density, s=7)
    
    ax[i].set_title(title[i], size=12)
    ax[1].set_xlabel('Site index', size=9)
    ax[i].set_ylabel('Excitation density', size=9)
    ax[i].set_ylim(-0.05, 1.05)

# fig.savefig(fname='low_det_pbc.png')
'''

"\nlat1 = lattice(11, legs=1, bc='OBC', constraint=False)\nlat2 = lattice(13, legs=1, bc='OBC', constraint=False)\n\nmat = []\nmat.append(matrix(lat1, num=-4, rabi=1/2, int=4.5**6))\nmat.append(matrix(lat2, num=-4, rabi=1/2, int=5.5**6))\n\nfig, ax = plt.subplots(nrows=1, ncols=2, dpi=200, figsize=(8,2.5))\n\ntitle = [r'$Z_5$', r'$Z_6$']\n\nfor i in range(2):\n    density = [np.sum([mat[i].gs[k]**2 * mat[i].basis[k][0,x] for k in range(len(mat[i].basis))]) for x in range(mat[i].sites)]\n    ax[i].plot(density)\n    ax[i].scatter(range(len(density)), density, s=7)\n    \n    ax[i].set_title(title[i], size=12)\n    ax[1].set_xlabel('Site index', size=9)\n    ax[i].set_ylabel('Excitation density', size=9)\n    ax[i].set_ylim(-0.05, 1.05)\n\n# fig.savefig(fname='low_det_pbc.png')\n"

In [26]:
'''
L=21
lat = lattice(L, legs=1, bc='OBC', constraint=True)
mat = []

det = [-1/2,0,1/2,1]
cols = len(det)

for j in range(cols):
    mat.append(matrix(lattice=lat, det=-det[j], rabi=1/2))
'''

"\nL=21\nlat = lattice(L, legs=1, bc='OBC', constraint=True)\nmat = []\n\nnum = [-1/2,0,1/2,1]\ncols = len(num)\n\nfor j in range(cols):\n    mat.append(matrix(lattice=lat, num=-num[j], rabi=1/2))\n"

In [27]:
'''
fig, ax = plt.subplots(nrows=2, ncols=2, dpi=200, figsize=(8,5))

for i,j in product([0,1],[0,1]):
    density = [np.sum([mat[2*i+j].gs[k]**2 * mat[2*i+j].basis[k][0,x] for k in range(len(mat[2*i+j].basis))]) for x in range(mat[2*i+j].sites)]
    ax[i,j].plot(density)
    ax[i,j].scatter(range(len(density)), density, s=7)
    
    ax[i,j].set_title(rf'$\Delta={det[2*i+j]}$', size=12)
    ax[1,j].set_xlabel('Site index', size=9)
    ax[i,0].set_ylabel('Excitation density', size=9)
    ax[i,j].set_ylim(-0.05, 1.05)

    # rabi = [mat[j].gs.T @ lat.r_rabi[0,x] @ mat[j].gs for x in range(mat[j].sites)]
    # ax[1,j].plot(rabi)
    # ax[1,j].scatter(range(len(rabi)), rabi, s=7)
    # ax[1,j].set_ylim(-0.55, 0.05)

# ax[1].set_ylim(-0.05, .55)
# ax[2].set_ylim(-0.05, .55)


# ax[0].set_yticks([0,0.1,0.2,0.3,0.4,0.5], labels=[0,0.1,0.2,0.3,0.4,0.5])
# ax[0].set_xticks(np.arange(0, L, int(np.round(L/5))), labels=np.arange(0, L, int(np.round(L/5))))
# ax[1].set_xticks(np.arange(0, L, int(np.round(L/5))), labels=np.arange(0, L, int(np.round(L/5))))
# ax[2].set_xticks(np.arange(0, L, int(np.round(L/5))), labels=np.arange(0, L, int(np.round(L/5))))

# fig.suptitle('Test')


# fig.savefig(fname='z_n.png', dpi=200)
'''

"\nfig, ax = plt.subplots(nrows=2, ncols=2, dpi=200, figsize=(8,5))\n\nfor i,j in product([0,1],[0,1]):\n    density = [np.sum([mat[2*i+j].gs[k]**2 * mat[2*i+j].basis[k][0,x] for k in range(len(mat[2*i+j].basis))]) for x in range(mat[2*i+j].sites)]\n    ax[i,j].plot(density)\n    ax[i,j].scatter(range(len(density)), density, s=7)\n    \n    ax[i,j].set_title(rf'$\\Delta={num[2*i+j]}$', size=12)\n    ax[1,j].set_xlabel('Site index', size=9)\n    ax[i,0].set_ylabel('Excitation density', size=9)\n    ax[i,j].set_ylim(-0.05, 1.05)\n\n    # rabi = [mat[j].gs.T @ lat.r_rabi[0,x] @ mat[j].gs for x in range(mat[j].sites)]\n    # ax[1,j].plot(rabi)\n    # ax[1,j].scatter(range(len(rabi)), rabi, s=7)\n    # ax[1,j].set_ylim(-0.55, 0.05)\n\n# ax[1].set_ylim(-0.05, .55)\n# ax[2].set_ylim(-0.05, .55)\n\n\n# ax[0].set_yticks([0,0.1,0.2,0.3,0.4,0.5], labels=[0,0.1,0.2,0.3,0.4,0.5])\n# ax[0].set_xticks(np.arange(0, L, int(np.round(L/5))), labels=np.arange(0, L, int(np.round(L/5))))\n# ax[1].set_xticks

In [28]:
'''
L=21
lat = lattice(L, legs=1, bc='PBC', constraint=True)
mat = []

det = [0,1/2]
cols = len(det)

for j in range(cols):
    mat.append(matrix(lattice=lat, det=-det[j], rabi=1/2))
'''

"\nL=21\nlat = lattice(L, legs=1, bc='PBC', constraint=True)\nmat = []\n\nnum = [0,1/2]\ncols = len(num)\n\nfor j in range(cols):\n    mat.append(matrix(lattice=lat, num=-num[j], rabi=1/2))\n"

In [29]:
'''
fig, ax = plt.subplots(nrows=1, ncols=2, dpi=200, figsize=(8,2.5))

for i in range(2):
    density = [np.sum([mat[i].gs[k]**2 * mat[i].basis[k][0,x] for k in range(len(mat[i].basis))]) for x in range(mat[i].sites)]
    ax[i].plot(density)
    ax[i].scatter(range(len(density)), density, s=7)
    
    ax[i].set_title(rf'$\Delta={det[i]}$', size=12)
    ax[1].set_xlabel('Site index', size=9)
    ax[i].set_ylabel('Excitation density', size=9)
    ax[i].set_ylim(-0.05, 1.05)

# fig.savefig(fname='low_det_pbc.png')
'''

"\nfig, ax = plt.subplots(nrows=1, ncols=2, dpi=200, figsize=(8,2.5))\n\nfor i in range(2):\n    density = [np.sum([mat[i].gs[k]**2 * mat[i].basis[k][0,x] for k in range(len(mat[i].basis))]) for x in range(mat[i].sites)]\n    ax[i].plot(density)\n    ax[i].scatter(range(len(density)), density, s=7)\n    \n    ax[i].set_title(rf'$\\Delta={num[i]}$', size=12)\n    ax[1].set_xlabel('Site index', size=9)\n    ax[i].set_ylabel('Excitation density', size=9)\n    ax[i].set_ylim(-0.05, 1.05)\n\n# fig.savefig(fname='low_det_pbc.png')\n"

In [30]:
'''
L=21
lat = lattice(L, legs=1, bc='OBC', constraint=True)
mat = []

rabi = [1/2, 1/2, 2, 0]
hop = [0, 1, 1, 1]

for j in range(4):
    mat.append(matrix(lattice=lat, rabi=rabi[j], hop=hop[j]))
'''

"\nL=21\nlat = lattice(L, legs=1, bc='OBC', constraint=True)\nmat = []\n\nrabi = [1/2, 1/2, 2, 0]\nhop = [0, 1, 1, 1]\n\nfor j in range(4):\n    mat.append(matrix(lattice=lat, rabi=rabi[j], hop=hop[j]))\n"

In [31]:
'''
fig, ax = plt.subplots(nrows=2, ncols=2, dpi=200, figsize=(8,5))

title = [r'$\Omega\gg t$', r'$\Omega=t$', r'$\Omega=4t$', r'$\Omega\ll t$']

for i,j in product([0,1],[0,1]):
    density = [np.sum([mat[2*i+j].gs[k]**2 * mat[2*i+j].basis[k][0,x] for k in range(len(mat[2*i+j].basis))]) for x in range(mat[2*i+j].sites)]
    ax[i,j].plot(density)
    ax[i,j].scatter(range(len(density)), density, s=7)
    
    ax[i,j].set_title(title[2*i+j])
    ax[1,j].set_xlabel('Site index', size=9)
    ax[i,0].set_ylabel('Excitation density', size=9)
    ax[i,j].set_ylim(-0.05, .55)

fig.savefig(fname='zigzag.png', dpi=200)
'''

"\nfig, ax = plt.subplots(nrows=2, ncols=2, dpi=200, figsize=(8,5))\n\ntitle = [r'$\\Omega\\gg t$', r'$\\Omega=t$', r'$\\Omega=4t$', r'$\\Omega\\ll t$']\n\nfor i,j in product([0,1],[0,1]):\n    density = [np.sum([mat[2*i+j].gs[k]**2 * mat[2*i+j].basis[k][0,x] for k in range(len(mat[2*i+j].basis))]) for x in range(mat[2*i+j].sites)]\n    ax[i,j].plot(density)\n    ax[i,j].scatter(range(len(density)), density, s=7)\n    \n    ax[i,j].set_title(title[2*i+j])\n    ax[1,j].set_xlabel('Site index', size=9)\n    ax[i,0].set_ylabel('Excitation density', size=9)\n    ax[i,j].set_ylim(-0.05, .55)\n\nfig.savefig(fname='zigzag.png', dpi=200)\n"

In [32]:
'''
L = 5
lat2 = lattice(L)

fig, ax = plt.subplots(nrows=1, ncols=1, dpi=200, figsize=(10/3, 2))

mat = matrix(lattice=lat2, hop=1/2, int=0)
density = [np.sum([mat.gs[k]**2 * mat.basis[k][0,x] for k in range(len(mat.basis))]) for x in range(mat.sites)]
ax.plot(np.array(density))
ax.scatter(range(len(density)), np.array(density), s=7)

ax.set_title(r'$\Omega=1$', size=12)
# ax[1].set_title(r'$\Omega = 4t$', size=12)
# ax[2].set_title(r'$\Omega \ll t$', size=12)
# ax.set_ylim(-0.05, .55)
# ax[1].set_ylim(-0.05, .55)
# ax[2].set_ylim(-0.05, .55)

ax.set_xlabel('Site index', size=9)
# ax[1].set_xlabel('Site index', size=9)
# ax[2].set_xlabel('Site index', size=9)

# ax.set_xticks(np.arange(0, L, int(np.round(L/5))), labels=np.arange(0, L, int(np.round(L/5))))

# fig.suptitle('Test')
ax.set_ylabel('Excitation density', size=9)
'''

"\nL = 5\nlat2 = lattice(L)\n\nfig, ax = plt.subplots(nrows=1, ncols=1, dpi=200, figsize=(10/3, 2))\n\nmat = matrix(lattice=lat2, hop=1/2, int=0)\ndensity = [np.sum([mat.gs[k]**2 * mat.basis[k][0,x] for k in range(len(mat.basis))]) for x in range(mat.sites)]\nax.plot(np.array(density))\nax.scatter(range(len(density)), np.array(density), s=7)\n\nax.set_title(r'$\\Omega=1$', size=12)\n# ax[1].set_title(r'$\\Omega = 4t$', size=12)\n# ax[2].set_title(r'$\\Omega \\ll t$', size=12)\n# ax.set_ylim(-0.05, .55)\n# ax[1].set_ylim(-0.05, .55)\n# ax[2].set_ylim(-0.05, .55)\n\nax.set_xlabel('Site index', size=9)\n# ax[1].set_xlabel('Site index', size=9)\n# ax[2].set_xlabel('Site index', size=9)\n\n# ax.set_xticks(np.arange(0, L, int(np.round(L/5))), labels=np.arange(0, L, int(np.round(L/5))))\n\n# fig.suptitle('Test')\nax.set_ylabel('Excitation density', size=9)\n"

In [33]:
'''
lat = lattice(15)

fig, ax = plt.subplots(nrows=1, ncols=1, dpi=200, figsize=(10/3, 2))

mat = matrix(lattice=lat, det=-1/2,rabi=1/2)
density = [np.sum([mat.gs[k]**2 * mat.basis[k][0,x] for k in range(len(mat.basis))]) for x in range(mat.sites)]
ax.plot(density)
ax.scatter(range(len(density)), density, s=7)

ax.set_title(r'$\Omega=1$', size=12)
ax.set_ylim(-0.05, 1.05)

ax.set_xlabel('Site index', size=9)
ax.set_ylabel('Excitation density', size=9)
'''

"\nlat = lattice(15)\n\nfig, ax = plt.subplots(nrows=1, ncols=1, dpi=200, figsize=(10/3, 2))\n\nmat = matrix(lattice=lat, num=-1/2,rabi=1/2)\ndensity = [np.sum([mat.gs[k]**2 * mat.basis[k][0,x] for k in range(len(mat.basis))]) for x in range(mat.sites)]\nax.plot(density)\nax.scatter(range(len(density)), density, s=7)\n\nax.set_title(r'$\\Omega=1$', size=12)\nax.set_ylim(-0.05, 1.05)\n\nax.set_xlabel('Site index', size=9)\nax.set_ylabel('Excitation density', size=9)\n"